In [2]:
import os
from unstructured.partition.pdf import partition_pdf

In [5]:
data_path = "./DATA/"
myfile = "annualreport.pdf"

elements = partition_pdf(filename = data_path+myfile,
              strategy = "hi_res",
              #extract_images_in_pdf = True,
              #extract_image_block_types = ["Image","Table"],
              #extract_image_block_to_payload = False, 
              #extract_image_block_output_dir = data_path+"images")
)
elements

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


 ...]

In [23]:
for e in elements: 
    if "unstructured.documents.elements.Table" in str(type(e)):
        print(e)

<h2>PyMuPDF</h2>

In [24]:
import io
import fitz
from PIL import Image

In [25]:
file = "./DATA/annualreport.pdf"

pdf_file = fitz.open(file)

In [46]:
#listar imagenes de la pagina 0:
pdf_file[0].get_images()

#DE la primera imagen que se ve en la pagina obtener su XREF (identificador)
pdf_file[0].get_images()[0][0]

#para extraer los bytes de la imagen: 
#1 sacamos la imagen base
xref = pdf_file[0].get_images()[0][0]
pdf_file.extract_image(xref)
print(pdf_file.extract_image(xref).keys())
#2 sacamos los bytes a traves de la imagen base
pdf_file.extract_image(xref)['image']
#extension (jpg, png, etc.)
pdf_file.extract_image(xref)['ext']
#Para guardarla en PILLOW:
Image.open(io.BytesIO(pdf_file.extract_image(xref)['image']))


with open('./DATA/images/image1.jpeg',"wb") as f:
    Image.open(io.BytesIO(pdf_file.extract_image(xref)['image'])).save(f)

dict_keys(['ext', 'smask', 'width', 'height', 'colorspace', 'bpc', 'xres', 'yres', 'cs-name', 'image'])


In [48]:
for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.get_images()

    if image_list:
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    else:
        print("[!] No images found on page", page_index)

    for image_index, img in enumerate(page.get_images(), start=1):
        xref = img[0]
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        image = Image.open(io.BytesIO(image_bytes))
        with open(f"./DATA/images/image{page_index+1}_{image_index}.{image_ext}", "wb") as f:
            image.save(f)

[+] Found a total of 2 images in page 0
[+] Found a total of 1 images in page 1
[+] Found a total of 6 images in page 2
[+] Found a total of 17 images in page 3
[+] Found a total of 2 images in page 4
[+] Found a total of 1 images in page 5
[+] Found a total of 1 images in page 6
[+] Found a total of 2 images in page 7
[+] Found a total of 5 images in page 8
[+] Found a total of 2 images in page 9
[+] Found a total of 2 images in page 10
[+] Found a total of 5 images in page 11
[+] Found a total of 1 images in page 12
[+] Found a total of 1 images in page 13
[+] Found a total of 6 images in page 14
[+] Found a total of 2 images in page 15
[+] Found a total of 1 images in page 16
[!] No images found on page 17
[+] Found a total of 5 images in page 18
[+] Found a total of 1 images in page 19
[+] Found a total of 4 images in page 20
[+] Found a total of 7 images in page 21
[+] Found a total of 1 images in page 22
[!] No images found on page 23
[+] Found a total of 4 images in page 24
[!] 

<h2> Extracción de tablas de un pdf </h2>

In [59]:
#Buscar pdf que tenga tablas 
import camelot

def get_tables(data, pages):
    for page in pages:
        table_list = camelot.read_pdf(data, pages = str(page))
        if table_list.n > 0: 
            for tab in range(table_list.n):
                #pasar tablas a df
                table_df = table_list[tab].df
                #table_df = (
                #    table_df.rename(columns = table_df.iloc[0]).drop(table_df.index[0]).reset_index(drop = True)
                #)
                #table_df = table_df.apply(lambda x: x.str.replace("\n",""))

                #table_df.columns = [col.replace('\n',' ').replace(' ','') for col in table_df.columns]
                #table_df.columns = [col.replace('(', '').replace(')', '') for col in table_df.columns]
                return table_df


In [65]:
data_path = './DATA/TABLAS_PDF.pdf'

df = get_tables(data_path, [7])
df


,0,1,2,3
0,,,,
1,,,,
2,,,,
3,,,,


In [63]:
data_json = df.to_json(orient='records')
data_json 

'[{"0":"","1":"","2":"","3":""},{"0":"","1":"","2":"","3":""},{"0":"","1":"","2":"","3":""},{"0":"","1":"","2":"","3":""}]'

<h3>PARENT-CHILD STRATEGY</h3>

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents.base import Document

from langchain_community.vectorstores import Chroma

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.vectorstores import VectorStore 
from langchain.load import dumps, loads
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List

In [2]:
llm = Ollama(base_url="http://localhost:11434",
             model="mistral",
             verbose=True,  
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

hugg_embeddings = HuggingFaceEmbeddings(model_name= "mxbai-embed-large-v1")

hugg_emb_bgem3 = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
myFile = 'annualreport.pdf'
loader = PyPDFLoader("DATA/" + myFile)

data = loader.load()

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    length_function = len
)

child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        length_function=len
    )

In [5]:
vectorstore = Chroma(
        collection_name="documents",
        embedding_function=hugg_embeddings
    )

store = InMemoryStore()

full_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

full_retriever.add_documents(data, ids = None)

In [6]:
vectorstore_bgem3 = Chroma(
        collection_name="documents",
        embedding_function=hugg_emb_bgem3
    )

#store_bge = InMemoryStore()

full_retriever_bge = ParentDocumentRetriever(
    vectorstore=vectorstore_bgem3,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

full_retriever_bge.add_documents(data, ids = None)

In [10]:
#Childs Splits 
vectorstore.similarity_search('Who is  Haroon Sheikh?', k = 4)

[Document(page_content='with his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment', metadata={'doc_id': '9e2a47ce-12e3-4db1-9c87-d99a3ac93d44', 'page': 40, 'source': 'DATA/annualreport.pdf'}),
 Document(page_content='facilitates a culture of openness and debate.\nGroup Chief Executive Officer\nHaroon Sheikh is the Group CEO and \naccountable to the Board for the day-to-day \nrunning of the Group and management of  \nthe strategic plan.\nHaroon Sheikh is responsible for the following:\n –Executive leadership of the Group’s \nbusiness on a day-to-day basis.\n –Developing the overall commercial', metadata={'doc_id': '16553046-7422-4056-a750-aa4374b1445

In [8]:
full_retriever.get_relevant_documents('Who is  Haroon Sheikh?')[0].page_content

[Document(page_content="and Robert W. Baird for two years until \nJune 2002. Karl set up Ashling Capital \nLLP in December 2002 to provide \nconsultancy services to quoted \nand private companies. He sits on a \nnumber of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment \nand passion for delivering high-quality \ncare and support to people with  \ncomplex needs. \nHaroon is Patron and Enterprise Fellow of \nthe Prince’s

In [28]:
template = """
    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    It is mandatorian that only if the answer is not in the context, answer "I have not enough context in order to answer this" and stop the answer.
    Try to use the memory context in the answer only if the question mentions it.
"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever= full_retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)
query = 'Is it true that Haroon Sheikh is the CEO of CareTech?'
result = qa_chain({"query": query})
result



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are a knowledgeable chatbot, here to help with questions of the user.
    Your tone should be professional and informative.

    Context: and Robert W. Baird for two years until 
June 2002. Karl set up Ashling Capital 
LLP in December 2002 to provide 
consultancy services to quoted 
and private companies. He sits on a 
number of AIM quoted and private 
company boards.January 2020 October 2005
Haroon is amongst the most experienced 
CEOs in the health and social care 
sector and one of the UK’s leading 
entrepreneurs and philanthropists. Along 
with his brother Farouq, he co-founded 
CareTech. As Group CEO he actively leads 
the day-to-day running of the Group 
and its international expansion, and has 
been instrumental in assembling a highly 
talented leadership team, to support 
the continued growth of the business. 
Haroon brings com

{'query': 'Is it true that Haroon Sheikh is the CEO of CareTech?',
 'result': ' Yes, Haroon Sheikh is the Group CEO of CareTech. He plays a significant role in leading the day-to-day operations and international expansion of the company. His experience, industry knowledge, commercial acumen, and deep commitment to delivering high-quality care have been instrumental in the growth and success of CareTech.'}

In [12]:
query = 'Is it true that Haroon Sheikh is the CEO of CareTech?'
sub_docs = vectorstore.similarity_search(query,k=3)
sub_docs2 = vectorstore_bgem3.similarity_search(query, k=3)
for d in sub_docs2:
    sub_docs.append(d)
#sub_docs.append(sub_docs2)
sub_docs

[Document(page_content='number of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has', metadata={'doc_id': '24887aaf-6851-403e-855e-288d7c438425', 'page': 40, 'source': 'DATA/annualreport.pdf'}),
 Document(page_content='with his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment', metadata={'doc_id': '24887aaf-6851-403e-

In [14]:
#Para el retriever personalizado que habrá que hacer cuando se unan todas las estrategias: 
#Esto es lo que la función get_relevant_document hace por debajo en la clase del Parent Retriever

#Hay una nota en la libreta sobre una idea que podria funcionar a la hora de meter esta técnica con el fucrag
query = 'Is it true that Haroon Sheikh is the CEO of CareTech?'
#sub_docs = vectorstore.similarity_search(query,k=3)
#sub_docs2 = vectorstore_bgem3.similarity_search(query, k=3)

ids = []
for d in sub_docs: 
    if d.metadata['doc_id'] not in ids: 
        ids.append(d.metadata['doc_id'])

parent_docs = store.mget(ids)

str_docs = []
docs = []
for pd in parent_docs:
    parent_doc_str = dumps(pd)
    if parent_doc_str not in str_docs:
        if pd is not None:
            str_docs.append(parent_doc_str)
            docs.append(loads(parent_doc_str))

docs

/home/joseacentic/centic/LLM/envPRUEBA/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[Document(page_content="and Robert W. Baird for two years until \nJune 2002. Karl set up Ashling Capital \nLLP in December 2002 to provide \nconsultancy services to quoted \nand private companies. He sits on a \nnumber of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment \nand passion for delivering high-quality \ncare and support to people with  \ncomplex needs. \nHaroon is Patron and Enterprise Fellow of \nthe Prince’s

In [16]:
dumps(vectorstore.similarity_search(query)[0])

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"page_content": "number of AIM quoted and private \\ncompany boards.January 2020 October 2005\\nHaroon is amongst the most experienced \\nCEOs in the health and social care \\nsector and one of the UK\\u2019s leading \\nentrepreneurs and philanthropists. Along \\nwith his brother Farouq, he co-founded \\nCareTech. As Group CEO he actively leads \\nthe day-to-day running of the Group \\nand its international expansion, and has", "metadata": {"doc_id": "24887aaf-6851-403e-855e-288d7c438425", "page": 40, "source": "DATA/annualreport.pdf"}}}'

In [ ]:
#hacer una prueba con una tecnica solo por ejemplo, L2 + cos y usar el parent child
#Si esto funciona extrapolarlo al completo sería fácil
class CustomRetriever(BaseRetriever):
    vs: VectorStore

    def rrf(self,fused_scores):
        reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
        
        lista_rerank = []
        print('Documentos finales (score)')
        for doc, score in reranked_results.items():
            lista_rerank.append(loads(doc))
            print('------')
            print(score)
        return lista_rerank
    
    def get_fused_scores(self,previus_fused, lista):
        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)
        return previus_fused
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
        ) -> List[Document]:
        
        docs_l2 = self.vs.similarity_search(query)
        docs_cos = self.vs.similarity_search_by_vector(hugg_embeddings.embed_query(query))

        scores_l2 = self.get_fused_scores({},docs_l2)
        scores_cos = self.get_fused_scores(scores_l2,docs_cos)

        rrf_docs = self.rrf(scores_cos)

        ids = []

        for d in rrf_docs: 
            if d.metadata['doc_id'] not in ids: 
                ids.append(d.metadata['doc_id'])

        parent_docs = store.mget(ids)

        final_docs = []
        for pd in parent_docs:
            if pd is not None:
                final_docs.append(pd)

        return final_docs

en la lista dada, intentar actualizar el metadata para tener en la lista el mismo metadata si el page_content es el mismo

In [21]:
#pruebas
def rrf(fused_scores):
    
    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
        
    lista_rerank = []
    print('Documentos finales (score)')
    for doc, score in reranked_results.items():
        lista_rerank.append(loads(doc))
        print('------')
        print(score)
    #Reranked_results nos da un diccionario {'doc_string':score, 'doc_str':score2,'doc_str':score3}
    print(reranked_results)
    return lista_rerank

def get_fused_scores(previus_fused, lista):
    for rank, doc in enumerate(lista):
        doc_str = dumps(doc)
        if doc_str not in previus_fused:
            previus_fused[doc_str] = 0
        previus_score = previus_fused[doc_str]
        print(f'scores previos {previus_score}')
        previus_fused[doc_str] += 1 / (rank + 1)
    return previus_fused

def get_fused_scores_v2(previus_fused, lista):
        
        previus_list = []
        for doc, score in previus_fused.items():
            previus_list.append(loads(doc))
        
        for d in lista:
            for p_d in previus_list:
                if d.page_content == p_d.page_content: 
                    d.metadata['doc_id'] = p_d.metadata['doc_id']

        for rank, doc in enumerate(lista):
            doc_str = dumps(doc)
            if doc_str not in previus_fused:
                previus_fused[doc_str] = 0
            previus_score = previus_fused[doc_str]
            print(f'scores previos {previus_score}')
            previus_fused[doc_str] += 1 / (rank + 1)

        return previus_fused



docs_l2 = vectorstore.similarity_search(query)
docs_cos = vectorstore.similarity_search_by_vector(hugg_embeddings.embed_query(query))
docs_bge = vectorstore_bgem3.similarity_search(query)

scores_l2 = get_fused_scores_v2({},docs_l2)
scores_cos = get_fused_scores_v2(scores_l2,docs_cos)

scores_n = get_fused_scores_v2(scores_cos,docs_bge)


rrf_docs = rrf(scores_n)

ids = []
for d in rrf_docs: 
    if d.metadata['doc_id'] not in ids: 
        ids.append(d.metadata['doc_id'])

parent_docs = store.mget(ids)

final_docs = []
for pd in parent_docs:
    if pd is not None:
        final_docs.append(pd)

final_docs

rrf_docs

scores previos 0
scores previos 0
scores previos 0
scores previos 0
scores previos 1.0
scores previos 0.5
scores previos 0.3333333333333333
scores previos 0.25
scores previos 2.0
scores previos 1.0
scores previos 0.5
scores previos 0
Documentos finales (score)
------
3.0
------
1.5
------
0.8333333333333333
------
0.6666666666666666
------
0.25
{'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "document", "Document"], "kwargs": {"page_content": "number of AIM quoted and private \\ncompany boards.January 2020 October 2005\\nHaroon is amongst the most experienced \\nCEOs in the health and social care \\nsector and one of the UK\\u2019s leading \\nentrepreneurs and philanthropists. Along \\nwith his brother Farouq, he co-founded \\nCareTech. As Group CEO he actively leads \\nthe day-to-day running of the Group \\nand its international expansion, and has", "metadata": {"doc_id": "24887aaf-6851-403e-855e-288d7c438425", "page": 40, "source": "DATA/annualreport.pdf"}}}': 3.0, '

[Document(page_content='number of AIM quoted and private \ncompany boards.January 2020 October 2005\nHaroon is amongst the most experienced \nCEOs in the health and social care \nsector and one of the UK’s leading \nentrepreneurs and philanthropists. Along \nwith his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has', metadata={'doc_id': '24887aaf-6851-403e-855e-288d7c438425', 'page': 40, 'source': 'DATA/annualreport.pdf'}),
 Document(page_content='with his brother Farouq, he co-founded \nCareTech. As Group CEO he actively leads \nthe day-to-day running of the Group \nand its international expansion, and has \nbeen instrumental in assembling a highly \ntalented leadership team, to support \nthe continued growth of the business. \nHaroon brings commercial acumen, \nrelated industry experience and property \nknowledge. He has a deep commitment', metadata={'doc_id': '24887aaf-6851-403e-